In [1]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input

# path to the model weights files.
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 1600

Using TensorFlow backend.


In [2]:
model =  Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(img_width, img_height, 3), activation='relu', border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

In [4]:
f = h5py.File(weights_path)
for k, layer_name in enumerate(f.attrs[u'layer_names']):
    if k == len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f[layer_name]
#    print layer_name,
#    print k, layer_name
    weights = [g[p] for p in g.keys()]
#    print weights[0].shape
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [5]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [6]:
# add the model on top of the convolutional base
model.add(top_model)

In [7]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:16]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-6, momentum=0.9),
              metrics=['accuracy'])

In [8]:
for layer in model.layers:
    print layer.name, 
    print layer.trainable

block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True
sequential_2 True


In [9]:
datagen = ImageDataGenerator(rescale=1., featurewise_center=True)
datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)


In [10]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1.,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        featurewise_center=True)
train_datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)


test_datagen = ImageDataGenerator(rescale=1.,
                                 featurewise_center=True)
test_datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)


train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [13]:
nb_epoch = 5
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples,
        verbose=2)

Epoch 1/5
63s - loss: 0.2449 - acc: 0.9840 - val_loss: 0.4409 - val_acc: 0.9712
Epoch 2/5
63s - loss: 0.2329 - acc: 0.9855 - val_loss: 0.4395 - val_acc: 0.9706
Epoch 3/5
63s - loss: 0.2481 - acc: 0.9840 - val_loss: 0.4422 - val_acc: 0.9706
Epoch 4/5
63s - loss: 0.3980 - acc: 0.9740 - val_loss: 0.4684 - val_acc: 0.9694
Epoch 5/5
63s - loss: 0.2403 - acc: 0.9845 - val_loss: 0.4360 - val_acc: 0.9712
